In [1]:
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
tf.__version__

'2.0.0'


3 - Building your first ResNet model (50 layers)
You now have the necessary blocks to build a very deep ResNet. The following figure describes in detail the architecture of this neural network. "ID BLOCK" in the diagram stands for "Identity block," and "ID BLOCK x3" means you should stack 3 identity blocks together.



The details of this ResNet-50 model are:

Zero-padding pads the input with a pad of (3,3)
Stage 1:
The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2). Its name is "conv1".
BatchNorm is applied to the channels axis of the input.
MaxPooling uses a (3,3) window and a (2,2) stride.
Stage 2:
The convolutional block uses three set of filters of size [64,64,256], "f" is 3, "s" is 1 and the block is "a".
The 2 identity blocks use three set of filters of size [64,64,256], "f" is 3 and the blocks are "b" and "c".
Stage 3:
The convolutional block uses three set of filters of size [128,128,512], "f" is 3, "s" is 2 and the block is "a".
The 3 identity blocks use three set of filters of size [128,128,512], "f" is 3 and the blocks are "b", "c" and "d".
Stage 4:
The convolutional block uses three set of filters of size [256, 256, 1024], "f" is 3, "s" is 2 and the block is "a".
The 5 identity blocks use three set of filters of size [256, 256, 1024], "f" is 3 and the blocks are "b", "c", "d", "e" and "f".
Stage 5:
The convolutional block uses three set of filters of size [512, 512, 2048], "f" is 3, "s" is 2 and the block is "a".
The 2 identity blocks use three set of filters of size [512, 512, 2048], "f" is 3 and the blocks are "b" and "c".
The 2D Average Pooling uses a window of shape (2,2) and its name is "avg_pool".
The flatten doesn't have any hyperparameters or name.
The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation. Its name should be 'fc' + str(classes).

In [3]:
def identity_block(input_data, k, filters, name):
    
    input_shortcut = input_data
    F1, F2, F3 = filters
    # main path
    x = keras.layers.Conv2D(F1, kernel_size=(1,1), strides=(1,1))(input_data)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)
    
    x = keras.layers.Conv2D(F2,kernel_size=(k,k), strides=(1,1), padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)
    
    x = keras.layers.Conv2D(F3, kernel_size=(1,1), strides=(1,1))(x)
    x = keras.layers.BatchNormalization()(x)
    
    
    x = keras.layers.Add()([x ,input_shortcut])
    x = keras.layers.Activation("relu")(x)
    
    return x
    

In [4]:
def conv_block(input_data, k, filters, name, s=2):
#     print("input_data shape: ", input_data.shape)
    F1, F2, F3 = filters
    x = keras.layers.Conv2D(F1, kernel_size=(1,1), strides=(s,s))(input_data)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)
    
#     print("Shape after first conv: ", x.shape)
    
    x = keras.layers.Conv2D(F2,kernel_size=(k,k), strides=(1,1), padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)
    
#     print("Shape after second conv: ", x.shape)
    
    x = keras.layers.Conv2D(F3, kernel_size=(1,1), strides=(1,1))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)
    
#     print("Shape after third conv: ", x.shape)
    
    #shortcut with conv
    shortcut = keras.layers.Conv2D(F3, kernel_size=(1,1), strides=(s,s))(input_data)
    shortcut = keras.layers.BatchNormalization()(shortcut)
    
#     print("Shape shortcut conv: ", shortcut.shape)
    
    x = keras.layers.Add()([x, shortcut])
    x = keras.layers.Activation("relu")(x)
    
    return x
    
    

In [5]:
def ResNet50(input_shape, classes):
    
    X_Input = keras.layers.Input(shape=input_shape)
#     print("X_Input: ", X_Input.shape)
    x = keras.layers.ZeroPadding2D((3,3))(X_Input)
#     print("x zero padded : ", x.shape)
    # STAGE 1
    x = keras.layers.Conv2D(64, (7,7), strides=(2,2), name="conv1")(x)
#     print("conv1 shape: ", x.shape)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)
    x = keras.layers.ZeroPadding2D((3,3))(X_Input)
    x = keras.layers.MaxPool2D((3,3), strides=(2,2))(x)
#     print("MaxPool2D shape: ", x.shape)
    
   
    
    #STAGE 2
    x = conv_block(x, k=3, filters=[64,64,256], s=1, name="2_a")
    x = identity_block(x, k=3, filters=[64,64,256], name="2_b")
    x = identity_block(x, k=3, filters=[64,64,256], name="2_c")
    
    #STAGE 3
    x = conv_block(x, k=3, filters=[128,128,512], name="3_a")
    x = identity_block(x, k=3, filters=[128,128,512], name="3_b")
    x = identity_block(x, k=3, filters=[128,128,512], name="3_c")
    x = identity_block(x, k=3, filters=[128,128,512], name="3_d")
    
    #STAGE 4
    x = conv_block(x, k=3, filters=[256,256,1024], name="4_a")
    x = identity_block(x, k=3, filters=[256,256,1024], name="4_b")
    x = identity_block(x, k=3, filters=[256,256,1024], name="4_c")
    x = identity_block(x, k=3, filters=[256,256,1024], name="4_d")
    x = identity_block(x, k=3, filters=[256,256,1024], name="4_e")
    x = identity_block(x, k=3, filters=[256,256,1024], name="4_f")
    
    #STAGE 5
    x = conv_block(x, k=3, filters=[512,512,2048], name="5_a")
    x = identity_block(x, k=3, filters=[512,512,2048], name="5_b")
    x = identity_block(x, k=3, filters=[512,512,2048], name="5_c")
    
    x = keras.layers.AveragePooling2D(strides=(2,2), name="avg_pool")(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(classes, activation="sigmoid")(x)
    
    # Create model
    model = keras.Model(inputs = X_Input, outputs = x, name='ResNet50')

    return model
    
    

In [6]:
model = ResNet50(input_shape=(32,32,3), classes=10)

In [7]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 18, 18, 3)    0           zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 18, 18, 64)   256         max_pooling2d[0][0]              
___________________________________________________________________________________________

In [8]:
base_model = keras.applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (32,32,3))

In [9]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [10]:
(train_images,train_labels), (test_images,test_labels) = keras.datasets.cifar10.load_data()

In [11]:
train_images.shape

(50000, 32, 32, 3)

In [12]:
train_images = train_images/255.0
test_images = test_images/255.0

In [13]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [14]:
model.fit(x=train_images, y=train_labels, validation_data=(test_images,test_labels), epochs=25)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 696s 14ms/sample - loss: 1.9215 - accuracy: 0.2656 - val_loss: 1.5863 - val_accuracy: 0.4081
Epoch 2/25
50000/50000 [==============================] - 686s 14ms/sample - loss: 1.3559 - accuracy: 0.5109 - val_loss: 1.2830 - val_accuracy: 0.5324
Epoch 3/25
50000/50000 [==============================] - 685s 14ms/sample - loss: 1.1058 - accuracy: 0.6079 - val_loss: 1.0963 - val_accuracy: 0.6149
Epoch 4/25
50000/50000 [==============================] - 684s 14ms/sample - loss: 0.9369 - accuracy: 0.6665 - val_loss: 1.0432 - val_accuracy: 0.6342
Epoch 5/25
50000/50000 [==============================] - 685s 14ms/sample - loss: 0.7905 - accuracy: 0.7226 - val_loss: 0.9491 - val_accuracy: 0.6766
Epoch 6/25
50000/50000 [==============================] - 685s 14ms/sample - loss: 0.6608 - accuracy: 0.7670 - val_loss: 1.0144 - val_accuracy: 0.6691
Epoch 7/25
50000/50000 [====================

In [16]:
model.fit?

In [17]:
base_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [20]:
sample_model = keras.Sequential([
    base_model,
#     keras.layers.AveragePooling2D(strides=(2,2), name="avg_pool"),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation="sigmoid")
])

In [21]:
sample_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [22]:
sample_model.fit(x=train_images, y=train_labels, validation_data=(test_images,test_labels), epochs=25)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 1011s 20ms/sample - loss: 1.9956 - accuracy: 0.3178 - val_loss: 1.6552 - val_accuracy: 0.3975
Epoch 2/25
50000/50000 [==============================] - 1003s 20ms/sample - loss: 1.8657 - accuracy: 0.3546 - val_loss: 1.7453 - val_accuracy: 0.3709
Epoch 3/25
50000/50000 [==============================] - 1003s 20ms/sample - loss: 1.7582 - accuracy: 0.3793 - val_loss: 1.5726 - val_accuracy: 0.4313
Epoch 4/25
50000/50000 [==============================] - 1003s 20ms/sample - loss: 1.7052 - accuracy: 0.3932 - val_loss: 1.5493 - val_accuracy: 0.4284
Epoch 5/25
50000/50000 [==============================] - 1004s 20ms/sample - loss: 1.5859 - accuracy: 0.4177 - val_loss: 1.4848 - val_accuracy: 0.4487
Epoch 6/25
50000/50000 [==============================] - 1003s 20ms/sample - loss: 1.5200 - accuracy: 0.4426 - val_loss: 1.3962 - val_accuracy: 0.4832
Epoch 7/25
50000/50000 [==============

KeyboardInterrupt: 

In [23]:
sample_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________
